In [259]:
import requests
import json
import pandas as pd
from collections import OrderedDict

#import fbbt
fbbt = requests.get("http://purl.obolibrary.org/obo/fbbt/2018-08-08/fbbt.json").json()
#fbbt.keys()

#compartments
c_infile = pd.read_csv("lb_compartments.tsv", sep="\t")
c_outfile = 'compartment_candidates.tsv'

#lineages
l_infile = pd.read_csv("lb_lineages.tsv", sep="\t")
l_outfile = 'lineage_candidates.tsv'

#fascicles
f_infile = pd.read_csv("lb_fascicles.tsv", sep="\t")
f_outfile = 'fascicle_candidates.tsv'


infile = f_infile
outfile = f_outfile
term_list = sorted(list(set(infile.larvalbrain_Name)))

term_list

['ADC',
 'ALC',
 'ALT',
 'ALT anterior entry',
 'BAl3_4',
 'BLP1_2',
 'DLT',
 'DMT',
 'DPLam',
 'DPMm1 to prFBpl',
 'DPMpm1_2 to prFBpl',
 'DPPT common',
 'DPPT descending',
 'DPPT post commissure',
 'FrMC',
 'GC',
 'MBDL',
 'MEF common',
 'MEF to great commissure',
 'MEF to lateral accessory lobe commissure',
 'MEF to primordium of posterior plexus of fan-shaped body',
 'SAC',
 'SuEC',
 'Subesophageal',
 'Supraesophageal',
 'VMT',
 'VMT to ventral central intermediate tract',
 'aLE',
 'aLEF',
 'aloSM',
 'antennal lobe commisssure',
 'antennal lobe tract',
 'anterior dorsal commissure',
 'anterior intermediate commissure of abdominal segment 1',
 'anterior intermediate commissure of abdominal segment 2',
 'anterior intermediate commissure of abdominal segment 3',
 'anterior intermediate commissure of abdominal segment 4',
 'anterior intermediate commissure of abdominal segment 5',
 'anterior intermediate commissure of abdominal segment 6',
 'anterior intermediate commissure of abdomina

In [260]:
def search_function(search_term: str):
    output_id=list()
    output_label=list()
    for n in fbbt['graphs'][0]['nodes']:
        try:
            test_result = True
            for word in str.lower(search_term).split(" "):
                if word not in str.lower(n['lbl']): test_result = False
                else: continue
            if test_result == True:
                output_id.append(n['id'])
                output_label.append(n['lbl'])
        except KeyError: continue
        try:
            for s in n['meta']['synonyms']:
                test_result = True
                for word in str.lower(search_term).split(" "):
                    if word not in str.lower(s['val']) or n['id'] in output_id: test_result = False
                    else: continue
                if test_result == True:
                    output_id.append(n['id'])
                    output_label.append(n['lbl'])
        except KeyError: continue
    return dict(zip(output_id,output_label))



In [261]:
mapping = dict()
for term in term_list:
    mapping[term] = search_function(term)
          

In [262]:
#mapping

In [263]:
map_table = pd.DataFrame(columns=('larvalbrain_Name','FBbt_id','FBbt_Name'))
row_od = OrderedDict([]) #new template row as an empty ordered dictionary

for key1 in mapping:
    if mapping[key1] == {}:
        row_od['larvalbrain_Name'] = key1
        row_od['FBbt_id'] = ""
        row_od['FBbt_Name'] = ""
        new_row = pd.DataFrame.from_records([row_od])
        map_table = pd.concat([map_table, new_row], ignore_index=True, sort=False)
    else:
        for key2 in mapping[key1]:
            row_od['larvalbrain_Name'] = key1
            row_od['FBbt_id'] = key2
            row_od['FBbt_Name'] = mapping[key1][key2]
            new_row = pd.DataFrame.from_records([row_od])
            map_table = pd.concat([map_table, new_row], ignore_index=True, sort=False)
    
#map_table

In [264]:
map_table.to_csv(outfile, sep = '\t')